In [1]:
import os
cwd = os.getcwd()
DB_FILE = "%s\Data\loans.db" % cwd

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing

In [3]:
##############################################################################
##########                DATABASE FUNCTIONS                     #############
##############################################################################
#### Read function to import data from the SQL to a pandas dataframe.
def readSQL(query):
    import pandas as pd
    import sqlite3 as sql3
    db = sql3.connect(DB_FILE)
    df = pd.read_sql_query(query, db)
    db.close()
    return(df)

#### Write a pandas dataframe into an SQL table. Use overwrite=True if you want to delete 
#### first a pre-existent table with the same name. Use append=True if you want to append
#### the data in the dataframe to a pre-existent table.
def writeSQL(df,tablename,overwrite=False, append=False):
    import pandas as pd
    import sqlite3 as sql
    db = sql.connect(DB_FILE)
    if (overwrite):
        action = "replace"
    elif (append):
        action = "append"
    else: 
        action = "fail"
    df.to_sql(tablename, db, if_exists=action)
    db.close()


In [4]:
loans = readSQL('''SELECT * FROM loans_dataset_missing''')

In [35]:
loans.head()

,verification_status,issue_d,pymnt_plan,purpose,initial_list_status,emp_length,loan_amnt,funded_amnt,funded_amnt_inv,term,...,annual_inc_rt,inc_grp,revol_bal_rt,credit_hist_log,credit_hist_rt,credit_hist_grp,max_L,max_fund_L,MAX_fund_L_inv,is_acc_now_delinq_not_zero
0,Not Verified,2007-10-01,0,debt_consolidation,1,2 years,10000,10000,2175.00,0,...,190.787840,verylow_inc,95.739229,2.397895,3.316625,4-13_yrs,0.0,0.0,0.0,0.0
1,Not Verified,2007-12-01,0,credit_card,1,1 year,5400,5400,375.00,0,...,204.939015,verylow_inc,195.074345,2.079442,2.828427,4-13_yrs,0.0,0.0,0.0,0.0
2,Not Verified,2007-11-01,0,debt_consolidation,1,2 years,1500,1500,583.05,0,...,124.899960,verylow_inc,79.448096,1.945910,2.645751,4-13_yrs,0.0,0.0,0.0,0.0
3,Not Verified,2007-12-01,0,home_improvement,1,6 years,4800,4800,1550.00,0,...,244.948974,low_inc,53.122500,2.397895,3.316625,4-13_yrs,0.0,0.0,0.0,0.0
4,Not Verified,2007-10-01,0,debt_consolidation,1,< 1 year,4800,4800,1625.00,0,...,158.113883,verylow_inc,20.223748,2.302585,3.162278,4-13_yrs,0.0,0.0,0.0,0.0


In [36]:
loans.dtypes

verification_status                object
issue_d                            object
pymnt_plan                         object
purpose                            object
initial_list_status                object
emp_length                         object
loan_amnt                           int64
funded_amnt                         int64
funded_amnt_inv                   float64
term                               object
collections_12_mths_ex_med          int64
home_ownership                     object
annual_inc                        float64
dti                               float64
delinq_2yrs                         int64
acc_now_delinq                      int64
inq_last_6mths                      int64
open_acc                            int64
pub_rec                             int64
revol_bal                         float64
revol_util                        float64
total_acc                           int64
default                            object
loan_installment                  

In [37]:
df = loans.copy()

In [38]:
df.drop(['issue_d'],axis=1,inplace=True)

In [39]:
##Get Series of columns
##cat_vars = list(df.select_dtypes(include=['object']).columns)
##cat_vars
##Turn
#for column in cat_vars:
 #   print(df[column])
 #   df[column] = df[column].astype('category')

In [40]:
## check unique - df.select_dtypes('category').apply(pd.Series.nunique, axis = 0)

<h3> Encoding the Categorical Variables

<b> For categorical variables with only 2 unique values we will use LabelEncoder

In [41]:
count = 0
for col in df:
    if df[col].dtype == 'object':
        if len(list(df[col].unique())) <= 2:     
            le = preprocessing.LabelEncoder()
            df[col] = le.fit_transform(df[col])
            count += 1
            print (col)
            
print('%d columns were label encoded.' % count)

pymnt_plan
initial_list_status
term
default
cat_collections_12_mths_ex_med
cat_pub_rec
6 columns were label encoded.


<b> For categorical variables with more than 2 unique values we will use dummies(onehotencoding)

In [42]:
df = pd.get_dummies(df)
print(df.shape)
df.head()

(254794, 138)


,pymnt_plan,initial_list_status,loan_amnt,funded_amnt,funded_amnt_inv,term,collections_12_mths_ex_med,annual_inc,dti,delinq_2yrs,...,inc_grp_high_inc,inc_grp_low_inc,inc_grp_middle_inc,inc_grp_veryhigh_inc,inc_grp_verylow_inc,credit_hist_grp_13-17_yrs,credit_hist_grp_17-21_yrs,credit_hist_grp_4-13_yrs,credit_hist_grp_<4_yrs,credit_hist_grp_>21_yrs
0,0,1,10000,10000,2175.00,0,0,36400.0,18.92,0,...,0,0,0,0,1,0,0,1,0,0
1,0,1,5400,5400,375.00,0,0,42000.0,16.09,0,...,0,0,0,0,1,0,0,1,0,0
2,0,1,1500,1500,583.05,0,0,15600.0,14.23,0,...,0,0,0,0,1,0,0,1,0,0
3,0,1,4800,4800,1550.00,0,0,60000.0,12.94,0,...,0,1,0,0,0,0,0,1,0,0
4,0,1,4800,4800,1625.00,0,0,25000.0,18.19,1,...,0,0,0,0,1,0,0,1,0,0


In [43]:
##check for NA
df.columns[df.isna().any()].tolist()

[]

In [44]:
##Turn Our Outcome To Category For TableOne

In [45]:
df['default'] = df['default'].astype('category')

# Feature Selection Strategy

<p>After the dataset is ready we need to find what features are good for modeling.</p>
<p>We will do it with Univariable and Multivariable methods.</p>
<p>For Univariate we will check with <strong>tableOne</strong> what features have <strong>p-value&lt;0.05</strong></p>
<p>For Multivariate we will check with numerous modeling:</p>
<ol>
<li>LASSO (L1 penalization)</li>
<li>Random Forest</li>
<li>Gradient Boosting classification</li>
<li>SVM classification</li>
</ol>

In [46]:
##Get Thomas Library
import pyMechkar as mechkar

<b>prepare the data for multiple checks

In [47]:
##Get All Columns except default
df_columns = df.select_dtypes(exclude=['category']).columns

In [48]:
X = df.loc[:,df_columns]
X.head()
X.shape

(254794, 137)

In [49]:
y = df[['default']]
print(y.shape)

(254794, 1)


Prepare table with all of our features and fill it with the result of each analysis method

In [53]:
varSelection = pd.DataFrame({'Variable': df_columns})
varSelection.size

137

<b>Univariable check

In [55]:
tab1 = mechkar.pyMechkar().Table1(data=df, y='default')

Factorizing... please wait
*****************************************************************************************************************************************
[]
[******************************************************************************************************************************************
[******************************************************************************************************************************************
['Unable to calcualte the Fisher exact test for variables acc_now_delinq and default... The p-value may be incorrect']
------ Finished in 521.6546809673309econds -----


In [56]:
tab1[tab1['p_value']<0.05]

,Variables,Categories,Population,Category_0,Category_1,p_value
4,initial_list_status,0,"184,189.00 ( 72.30%)","150,650.00 ( 72.10%)","33,539.00 ( 73.20%)",0.000
5,loan_amnt,Mean (SD),"13,505.40 (8,111.30)","13,295.90 (8,037.90)","14,461.00 (8,371.60)",0.000
8,funded_amnt,Mean (SD),"13,463.50 (8,088.80)","13,255.40 (8,016.30)","14,412.50 (8,345.60)",0.000
11,funded_amnt_inv,Mean (SD),"13,348.60 (8,122.10)","13,150.90 (8,045.70)","14,250.10 (8,403.10)",0.000
15,term,1,"56,425.00 ( 22.10%)","40,201.00 ( 19.20%)","16,224.00 ( 35.40%)",0.000
16,collections_12_mths_ex_med,Mean (SD),0.00 ( 0.10),0.00 ( 0.10),0.00 ( 0.10),0.001
19,annual_inc,Mean (SD),"71,702.70 (42,945.00)","73,300.00 (43,937.90)","64,418.60 (37,231.70)",0.000
22,dti,Mean (SD),16.50 ( 7.80),16.10 ( 7.70),18.40 ( 7.90),0.000
25,delinq_2yrs,Mean (SD),0.20 ( 0.70),0.20 ( 0.70),0.30 ( 0.80),0.000
29,acc_now_delinq,1,687.00 ( 0.30%),542.00 ( 0.30%),145.00 ( 0.30%),0.037


In [57]:
vn1 = tab1.loc[tab1['p_value']<0.05,'Variables'].unique()
print(len(vn1))
vn1

95


array(['initial_list_status', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'collections_12_mths_ex_med',
       'annual_inc', 'dti', 'delinq_2yrs', 'acc_now_delinq',
       'inq_last_6mths', 'open_acc', 'revol_bal', 'revol_util',
       'total_acc', 'loan_installment', 'acc_ratio', 'credit_hist',
       'poverty_rate_p', 'cat_collections_12_mths_ex_med', 'open_acc_rt',
       'total_acc_rt', 'annual_inc_log', 'annual_inc_rt', 'revol_bal_rt',
       'credit_hist_log', 'credit_hist_rt', 'max_L', 'max_fund_L',
       'MAX_fund_L_inv', 'is_acc_now_delinq_not_zero',
       'verification_status_Not Verified',
       'verification_status_Source Verified',
       'verification_status_Verified', 'purpose_car',
       'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_home_improvement', 'purpose_major_purchase',
       'purpose_medical', 'purpose_moving', 'purpose_other',
       'purpose_small_business', 'purpose_wedding',
       'emp_length_10+ years', 'emp_le

We will add these variables to our variable selection table

In [58]:
varSelection['Univarable'] = 0
varSelection.loc[varSelection['Variable'].isin(vn1), 'Univarable'] = 1
varSelection

,Variable,Univarable
0,pymnt_plan,0
1,initial_list_status,1
2,loan_amnt,1
3,funded_amnt,1
4,funded_amnt_inv,1
5,term,1
6,collections_12_mths_ex_med,1
7,annual_inc,1
8,dti,1
9,delinq_2yrs,1


<b>Multivariable Analysis checks

### Variable Selection using LASSO (L1 penalization)

In [59]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

lassomod = Lasso(alpha=0.1,max_iter=10000).fit(X, y)

In [60]:
model = SelectFromModel(lassomod, prefit=True)
#model.get_support()

In [61]:
varSelection['Lasso'] = model.get_support().astype('int64')
#varSelection

### Variable Selection using Random Forest

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

rfmod = RandomForestClassifier().fit(X, y.values.ravel())
#rfmod.feature_importances_ 

In [63]:
model = SelectFromModel(rfmod, prefit=True)
#model.get_support()

In [64]:
varSelection['RandomForest'] = model.get_support().astype('int64')
#varSelection

### Variable Selection using Gradient Boosting classification

In [65]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel

gbmod = GradientBoostingClassifier().fit(X, y.values.ravel())

In [66]:
model = SelectFromModel(gbmod, prefit=True)
#model.get_support()

In [67]:
varSelection['GradientBoost'] = model.get_support().astype('int64')
#varSelection

### Variable Selection using SVM classification

In [68]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

svmmod = LinearSVC(C=0.01, penalty="l1",dual=False).fit(X, y.values.ravel())

In [69]:
model = SelectFromModel(svmmod, prefit=True)
#model.get_support()

In [70]:
varSelection['SVM'] = model.get_support().astype('int64')
#varSelection

### Summarization and Selection of Variables 

In [71]:
varSelection['Sum'] =  np.sum(varSelection,axis=1)
varSelection

,Variable,Univarable,Lasso,RandomForest,GradientBoost,SVM,Sum
0,pymnt_plan,0,0,0,0,0,0
1,initial_list_status,1,0,1,1,1,4
2,loan_amnt,1,1,1,1,0,4
3,funded_amnt,1,1,1,1,1,5
4,funded_amnt_inv,1,1,1,1,1,5
5,term,1,0,1,1,1,4
6,collections_12_mths_ex_med,1,0,0,0,0,1
7,annual_inc,1,0,1,1,0,3
8,dti,1,1,1,1,1,5
9,delinq_2yrs,1,0,0,1,1,3


In [73]:
varSelection.groupby('Sum')['Variable'].count()


Sum
0    30
1    29
2    46
3    17
4     8
5     7
Name: Variable, dtype: int64

We can now decide a threshold for selecting our variables!

In [74]:
v=varSelection[varSelection['Sum']>1]

In [77]:
cols = v["Variable"]

In [80]:
cols = cols.append(pd.Series('default'))

In [81]:
df2 = df.loc[:,cols]

In [82]:
print(df2.shape)

(254794, 79)


In [83]:
writeSQL(df=df2,tablename="full_dataset")

C:\Users\Kobi\Anaconda3\lib\site-packages\pandas\core\generic.py:2127: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)
